In [ ]:
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from matplotlib.ticker import FormatStrFormatter

In [ ]:
import json
import os

In [ ]:
# JSON file containing the results for this simulation run
results_file = "/Users/brianmccarthy/git_repos/results-analysis/configs/cv2x.json"

In [ ]:
with open(results_file) as results_json:
    config = json.load(results_json)["cv2x"]

In [ ]:
# Markers to use for this run
markers = [".", "o", "v", "^", "<", ">", "1", "2", "3", "4", "8", "s", "p", "P", "*", "h",
           "H", "+", "x", "X", "D", "d", "|", "_", 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

In [ ]:
def create_bins(lower_bound, width, quantity):
    """ create_bins returns an equal-width (distance) partitioning.
        It returns an ascending list of tuples, representing the intervals.
        A tuple bins[i], i.e. (bins[i][0], bins[i][1])  with i > 0
        and i < quantity, satisfies the following conditions:
            (1) bins[i][0] + width == bins[i][1]
            (2) bins[i-1][0] + width == bins[i][0] and
                bins[i-1][1] + width == bins[i][1]
    """
    bins = []
    for low in range(lower_bound, lower_bound + quantity * width + 1, width):
        bins.append((low, low + width))
    return bins

In [ ]:
def bin_fields(df, fields, bin_width=10, bin_quantity=49):
    """
    Bins multiple dfs into a single dictionary that can be used as an average for multiple fields across multiple
    runs
    :param df: dataframe to bin
    :param fields: fields to be binned.
    :param bin_width: width of each bin
    :param bin_quantity: total number of bins
    :return:
    """
    bins = create_bins(lower_bound=0, width=bin_width, quantity=bin_quantity)
    distances = []
    overall_fields = {}
    for interval in bins:
        upper_b = interval[1]
        distances.append(upper_b)

    for field in fields:
        overall_fields[field] = []

    overall_fields["distance"] = distances

    distance_col = config["results"]["distance"]

    for i in range(len(bins)):
        lower_b = bins[i][0]
        upper_b = bins[i][1]
        fields_temp = df[(df[distance_col] >= lower_b) & (df[distance_col] < upper_b)]
        for field in fields:
            if i < len(overall_fields[field]):
                overall_fields[field][i] = (fields_temp[field].mean() + overall_fields[field][i]) / 2
            else:
                overall_fields[field].append(fields_temp[field].mean())

    return overall_fields

In [ ]:
def pdr_dist_individual(pdr, distances, label, plot_name, show=True, store=False):
    fig, ax = plt.subplots()

    ax.plot(distances, pdr, label=label, marker=markers[0], markevery=3)

    ax.set(xlabel='Distance (m)', ylabel='Packet Delivery Rate (PDR) %')
    ax.legend(loc='lower right')
    ax.tick_params(direction='in')

    ax.set_ylim([0, 100])
    plt.yticks(np.arange(0, 101, step=10))

    ax.set_xlim([0, (max(distances) + 1)])
    plt.xticks(np.arange(0, (max(distances) + 1), step=50))

    fig.suptitle(plot_name, fontsize=12)
    
    if show:
        fig.show()
        
    if store:
        fig.savefig("{}.png".format(plot_name), dpi=300)

In [ ]:
def pdr_dist(pdrs, distances, labels, plot_name, show=True, store=False):
    fig, ax = plt.subplots()

    for i in range(len(pdrs)):
        ax.plot(distances, pdrs[i], label=labels[i])

    ax.set(xlabel='Distance (m)', ylabel='Packet Delivery Rate (PDR) %')
    ax.legend(loc='lower left')
    ax.tick_params(direction='in')
    
    ax.set_ylim([0, 100])
    plt.yticks(np.arange(0, 101, step=10))

    ax.set_xlim([0, (max(distances) + 1)])
    plt.xticks(np.arange(0, (max(distances) + 1), step=50))

#     fig.suptitle(plot_name, fontsize=12)
    
    if show:
        fig.show()
        
    if store:
        fig.savefig("{}.png".format(plot_name), dpi=300)

In [ ]:
def errors_dist_individual(distances, decoded, errors, error_labels, plot_name, show=True, store=False):
    fig, ax = plt.subplots()

    ax.plot(distances, decoded, label="Decoded", marker=self.markers[0], markevery=3)

    for i in range(len(errors)):
        ax.plot(distances, errors[i], label=error_labels[i], marker=self.markers[i+1], markevery=3)

    ax.legend(loc='center left')
    ax.set(xlabel='Distance (m)', ylabel='Messages Decoded')
    ax.tick_params(direction='in')

    ax.set_ylim([0, 1])
    plt.yticks(np.arange(0, 1.1, step=.1))

    ax.set_xlim([0, (max(distances) + 1)])
    plt.xticks(np.arange(0, (max(distances) + 1), step=50))

    fig.suptitle(plot_name, fontsize=12)
    
    if show:
        fig.show()

    if store:
        fig.savefig("{}.png".format(plot_name), dpi=300)

In [ ]:
def errors_dist(distances, decoded, decoded_labels, errors, error_labels, plot_name, show=True, store=False):
    fig, ax = plt.subplots()

    for i in range(len(decoded)):
        ax.plot(distances, decoded[i], label=decoded_labels[i], marker=self.markers[i], markevery=3)

        for j in range(len(errors[i])):
            ax.plot(distances, errors[i][j], label=error_labels[i][j], marker=self.markers[i + j])

    ax.legend(loc='center left')
    ax.set(xlabel='Distance (m)', ylabel='Messages Decoded')
    ax.tick_params(direction='in')

    ax.set_ylim([0, 1])
    plt.yticks(np.arange(0, 1.1, step=.1))

    ax.set_xlim([0, (max(distances) + 1)])
    plt.xticks(np.arange(0, (max(distances) + 1), step=50))
    
    fig.suptitle(plot_name, fontsize=12)
    
    if show:
        fig.show()
        
    if store:
        fig.savefig("{}.png".format(plot_name), dpi=300)

In [ ]:
def box_plot_colour(data, labels, title, plot_name, colors = ['pink', 'lightblue', 'lightgreen'], show=True, store=False):
    fig, ax = plt.subplots()
    ax.set_title(title)
    bplot = ax.boxplot(data, labels=labels, patch_artist=True)

    for patch, color in zip(bplot['boxes'], colors):
        patch.set_facecolor(color)
    
    if show:
        fig.show()
        
    if store:
        fig.savefig("{}.png".format(plot_name), dpi=300)

In [ ]:
def box_plot(data, labels, title, plot_name, show=True, store=False):
    fig, ax = plt.subplots()
    ax.set_title(title)
    bplot = ax.boxplot(data, labels=labels)
    
    if show:
        fig.show()
        
    if store:
        fig.savefig("{}.png".format(plot_name), dpi=300)

In [ ]:
def ipg_dist(ipgs, distances, labels, plot_name, show=True, store=False):
    fig, ax = plt.subplots()

    for i in range(len(ipgs)):
        plt.scatter(distances, ipgs[i], marker=markers[i], label=labels[i])

    ax.set(xlabel='Distance (m)', ylabel='Inter Packet Gap (IPG) ms')
    ax.legend(loc='upper left')
    ax.tick_params(direction='in')
    
    ax.set_ylim([0, 1])
    plt.yticks(np.arange(0, 1.1, step=.1))

    ax.set_xlim([0, (max(distances) + 1)])
    plt.xticks(np.arange(0, (max(distances) + 1), step=50))

#     fig.suptitle(plot_name, fontsize=12)
    
    if show:
        fig.show()
        
    if store:
        fig.savefig("{}.png".format(plot_name), dpi=300)

### Parse the results into DF

- Periodic results
- Non-periodic results
- DCC results
- Non-DCC results

In [ ]:
def read_data_in(folder):
    first_file = True
    merged_df = pd.DataFrame()
    files = os.listdir(folder)
    files.sort()
    num_files = (len(files))
    count = 0
    for file in files:
        if count % 5 == 0:
            print("File {}/{}".format(count, num_files))
        if ".csv" in file:
            result_file = os.path.join(folder, file)
            if first_file:
                merged_df = pd.read_csv(result_file)
                first_file = False
            else:
                try:
                    merged_df.append(pd.read_csv(result_file))
                except pd.errors.ParserError as e:
                    print("Failed to parse file: {}".format(file))
        count+=1
    return merged_df

In [ ]:
interested_folder = ["After-1", "Random_Access_Low_Density", "Random_Access_Medium_Density", "Random_Access_Packet_Dropping_Low_Density", "Random_Access_Packet_Dropping_Medium_Density"]

In [ ]:
raw_data_folder = "/Users/brianmccarthy/git_repos/results-analysis/data/raw_data/cv2x"
loaded_dfs = {}

In [ ]:
for folder in os.listdir(raw_data_folder):
    if folder in interested_folder:
        print("="*50)
        print("Parsing configuration: {}".format(folder))
        print("="*50)
        loaded_dfs[folder] = read_data_in(os.path.join(raw_data_folder, folder))

In [ ]:
extra_folders = ["Random_Access_Packet_Dropping_Medium_Density"]
for folder in os.listdir(raw_data_folder):
    if folder in extra_folders:
        print("="*50)
        print("Parsing configuration: {}".format(folder))
        print("="*50)
        loaded_dfs[folder] = read_data_in(os.path.join(raw_data_folder, folder))

In [ ]:
distances = create_bins(0, 10, 49)

In [ ]:
for i in range(len(distances)):
    distances[i] = distances[i][1]

In [ ]:
pdrs = {}
ipgs = {}
for configuration in interested_folder:
    df = loaded_dfs[configuration]
    
    # Filter the times down
    df = df[df["Time"] > 502]
    
    # Put config back into our dictionary
    loaded_dfs[configuration] = df
    
    # Calculate pdr
    pdr = []
    ipg = []
    pdr_binned = bin_fields(df, ["tbDecoded"])
    ipg_binned = bin_fields(df, ["interPacketDelay"])
    
    for i in range(len(pdr_binned["tbDecoded"])):
        if i < len(pdr):
            pdr[i] = (pdr_binned["tbDecoded"][i] + pdr[i]) / 2
        else:
            pdr.append(pdr_binned["tbDecoded"][i])
            
    for i in range(len(ipg_binned["interPacketDelay"])):
        if i < len(ipg):
            ipg[i] = (ipg_binned["interPacketDelay"][i] + ipg[i]) / 2
        else:
            ipg.append(ipg_binned["interPacketDelay"][i])
            
    for i in range(len(pdr)):
        pdr[i] = pdr[i] * 100
    pdrs[configuration] = pdr
    
    ipgs[configuration] = ipg

In [ ]:
random = []
random.append(pdrs["After-1"])
random.append(pdrs["Random_Access_Low_Density"])
random.append(pdrs["Random_Access_Packet_Dropping_Low_Density"])
random.append(pdrs["Random_Access_Medium_Density"])
random.append(pdrs["Random_Access_Packet_Dropping_Medium_Density"])

In [ ]:
pdr_dist(random, distances, ["After-1", "Random_Access_Low_Density", "Random_Access_Packet_Dropping_Low_Density", "Random_Access_Medium_Density", "Random_Access_Packet_Dropping_Medium_Density"], "random_pdr", show=True, store=True)


In [ ]:
motivational_pdr_graph = []
motivational_pdr_graph.append(pdrs["Highway-fast"])
motivational_pdr_graph.append(pdrs["NO-CC"])
motivational_pdr_graph.append(pdrs["NO-CC-high-density"])

dcc_pdr_medium = []
dcc_pdr_medium.append(pdrs["DCC-Enabled"])
dcc_pdr_medium.append(pdrs["NO-CC"])
dcc_pdr_medium.append(pdrs["After-1"])
dcc_pdr_medium.append(pdrs["After-5"])

dcc_pdr_high = []
dcc_pdr_high.append(pdrs["DCC-Enabled"])
dcc_pdr_high.append(pdrs["NO-CC-high-density"])
dcc_pdr_high.append(pdrs["After-1-high-density"])
dcc_pdr_high.append(pdrs["After-5-high-density"])

In [ ]:
pdr_dist(motivational_pdr_graph, distances, ["Highway Fast", "Medium Density", "High Density"], "Motivation_pdr", show=True, store=True)


In [ ]:
pdr_dist(dcc_pdr_medium, distances, ["DCC Access", "No Congestion Control", "3GPP DCC Mechanism (CBR) - Immediate new grant", "3GPP DCC Mechanism (CBR) - 5 missed trans b4 new grant"], "Medium_pdr", show=True, store=True)


In [ ]:
pdr_dist(dcc_pdr_high, distances, ["DCC Access", "No Congestion Control", "3GPP DCC Mechanism (CBR) - Immediate new grant", "3GPP DCC Mechanism (CBR) - 5 missed trans b4 new grant"], "High_pdr", show=True, store=True)


In [ ]:
motivational_ipg_graph = []
motivational_ipg_graph.append(ipgs["Highway-fast"])
motivational_ipg_graph.append(ipgs["NO-CC"])
motivational_ipg_graph.append(ipgs["NO-CC-high-density"])

dcc_ipg_medium = []
dcc_ipg_medium.append(ipgs["DCC-Enabled"])
dcc_ipg_medium.append(ipgs["NO-CC"])
dcc_ipg_medium.append(ipgs["After-1"])
dcc_ipg_medium.append(ipgs["After-5"])

dcc_ipg_high = []
dcc_ipg_high.append(ipgs["DCC-Enabled"])
dcc_ipg_high.append(ipgs["NO-CC-high-density"])
dcc_ipg_high.append(ipgs["After-1-high-density"])
dcc_ipg_high.append(ipgs["After-5-high-density"])

In [ ]:
ipg_dist(motivational_ipg_graph, distances, ["Highway Fast", "Medium Density", "High Density"], "Motivation_ipg", show=True)


In [ ]:
ipg_dist(dcc_ipg_medium, distances, ["DCC Access", "No Congestion Control", "3GPP DCC Mechanism (CBR) - Immediate new grant", "3GPP DCC Mechanism (CBR) - 5 missed trans b4 new grant"], "Medium_ipg", show=True, store=False)


In [ ]:
ipg_dist(dcc_ipg_high, distances, ["DCC Access", "No Congestion Control", "3GPP DCC Mechanism (CBR) - Immediate new grant", "3GPP DCC Mechanism (CBR) - 5 missed trans b4 new grant"], "High_ipg", show=True, store=False)


### CBR 

Comparing CBR levels in DCC vs Non-DCC scenarios

In [ ]:
loaded_dfs["DCC-Enabled"].plot(x="Time", y="cbr", kind="line")

In [ ]:
loaded_dfs["NO-CC"].plot(x="Time", y="cbr", kind="line")

In [ ]:
loaded_dfs["After-2"].plot(x="Time", y="cbr", kind="line")

In [ ]:
dcc_enabled_df  = loaded_dfs["dcc-enabled"]
dcc_disabled_df = loaded_dfs["dcc-disabled"]
periodic_df  = loaded_dfs["periodic-300ms"]
non_periodic_df = loaded_dfs["non-periodic"]

In [ ]:
node_50_dcc_enabled_df = dcc_enabled_df[(dcc_enabled_df["NodeID"] == 50) & (dcc_enabled_df["cbr"] >= 0.0)]
node_50_dcc_disabled_df = dcc_disabled_df[(dcc_disabled_df["NodeID"] == 50) & (dcc_disabled_df["cbr"] >= 0.0)]

In [ ]:
node_50_dcc_enabled_df.plot(x="Time", y="cbr", kind="line")

In [ ]:
node_50_dcc_disabled_df.plot(x="Time", y="cbr", kind="line")

In [ ]:
print("="* 27)
print("DCC Disabled Statistics")
print("mean  : {}".format(node_50_dcc_disabled_df["cbr"].mean()))
print("std   : {}".format(node_50_dcc_disabled_df["cbr"].std()))
print("median: {}".format(node_50_dcc_disabled_df["cbr"].median()))

print("="* 27)

print("DCC Enabled Statistics")
print("mean  : {}".format(node_50_dcc_enabled_df["cbr"].mean()))
print("std   : {}".format(node_50_dcc_enabled_df["cbr"].std()))
print("median: {}".format(node_50_dcc_enabled_df["cbr"].median()))
print("="* 27)

### Grant breaking

In [ ]:
print("Generated grants: {}".format(dcc_disabled_df["grantStartTime"].count()))
print("Broken grants: {}".format(dcc_disabled_df["grantBreak"].count()))

In [ ]:
# Higher due to the issue of starting at 502
print("Generated Grants: {}".format(dcc_enabled_df["grantStartTime"].count()))
print("Broken Grants: {}".format(dcc_enabled_df["grantBreakMissedTrans"].count()))

In [ ]:
print("Generated grants: {}".format(periodic_df["grantStartTime"].count()))
print("Broken grants: {}".format(periodic_df["grantBreak"].count()))

In [ ]:
# Higher due to the issue of starting at 502
print("Generated Grants: {}".format(non_periodic_df["grantStartTime"].count()))
print("Broken Grants: {}".format(non_periodic_df["grantBreakMissedTrans"].count()))

## Inter Packet Gap

In [ ]:
file_path_after_1 = "/Users/brianmccarthy/git_repos/results-analysis/data/raw_data/cv2x/After-1/run-1-2019-12-28-08_32_56.csv"
file_path_after_5 = "/Users/brianmccarthy/git_repos/results-analysis/data/raw_data/cv2x/After-5/run-1-2019-12-29-10_26_28.csv"
file_path_dcc_enabled = "/Users/brianmccarthy/git_repos/results-analysis/data/raw_data/cv2x/DCC-Enabled/run-1-2020-01-02-11_24_37.csv"

df_after_1 = pd.read_csv(file_path_after_1)
df_after_5 = pd.read_csv(file_path_after_5)
df_dcc_enabled = pd.read_csv(file_path_dcc_enabled)

df_after_1 = df_after_1[(df_after_1["Time"] >= 502.000) & (df_after_1["interPacketDelay"].notnull()) & (df_after_1["txRxDistanceTB"] <= 500)]
df_after_5 = df_after_5[(df_after_5["Time"] >= 502.000) & (df_after_5["interPacketDelay"].notnull()) & (df_after_5["txRxDistanceTB"] <= 500)]
df_dcc_enabled = df_dcc_enabled[(df_dcc_enabled["Time"] >= 502.000) & (df_dcc_enabled["interPacketDelay"].notnull()) & (df_dcc_enabled["txRxDistanceTB"] <= 500)]

In [ ]:
print("==============================")
print("========= After-1 ===========")
print("Mean: {}".format(df_after_1["interPacketDelay"].mean()))
print("Median: {}".format(df_after_1["interPacketDelay"].median()))
print("Lower Quartile: {}".format(df_after_1["interPacketDelay"].quantile(.25)))
print("Upper Quartile: {}".format(df_after_1["interPacketDelay"].quantile(.75)))
print("==============================")

print("==============================")
print("========= After-5 ===========")
print("Mean: {}".format(df_after_5["interPacketDelay"].mean()))
print("Median: {}".format(df_after_5["interPacketDelay"].median()))
print("Lower Quartile: {}".format(df_after_5["interPacketDelay"].quantile(.25)))
print("Upper Quartile: {}".format(df_after_5["interPacketDelay"].quantile(.75)))
print("==============================")

print("==============================")
print("======= DCC Enabled ==========")
print("Mean: {}".format(df_dcc_enabled["interPacketDelay"].mean()))
print("Median: {}".format(df_dcc_enabled["interPacketDelay"].median()))
print("Lower Quartile: {}".format(df_dcc_enabled["interPacketDelay"].quantile(.25)))
print("Upper Quartile: {}".format(df_dcc_enabled["interPacketDelay"].quantile(.75)))
print("==============================")

In [ ]:
delays = []
delays.append(df_after_1["interPacketDelay"].values)
delays.append(df_after_5["interPacketDelay"].values)
delays.append(df_dcc_enabled["interPacketDelay"].values)

labels = ["After-1", "After-5", "DCC-Enabled"]

In [ ]:
box_plot(delays, labels , "latency Box plot", "latency Box plot")